In [1]:
import jax.numpy as jnp
import numpy as np


import hydraxmpm as hdx
import chex

%load_ext autoreload
%autoreload 2

In [2]:




"""Test the linear shape function for top level container input."""
position_stack = jnp.array([[0.45, 0.21], [0.8, 0.4]])

config = hdx.MPMConfig(
    origin=[0.0, 0.0],
    end=[
        1.0,
        1.0,
    ],
    cell_size=0.1,
    num_points=len(position_stack),
    shapefunction_type="linear",
)
grid = hdx.GridStencilMap(config)

grid = grid.partition(position_stack)

particles = hdx.Particles(config, position_stack=position_stack)
shapefunction = hdx.LinearShapeFunction(config)

# shapefunction = shapefunction.update_from_particles(grid, particles)

# expected_shapef_stack = jnp.array(
#     [
#         [0.45000005, 0.04999995, 0.45000005, 0.04999995],
#         [1.0, 0.0, 0.0, 0.0],
#     ]
# )
# np.testing.assert_allclose(expected_shapef_stack, shapefunction.shapef_stack)

# np.testing.assert_allclose(
#     jnp.prod(shapefunction.shapef_stack.flatten(), axis=0), 0
# )

# expected_shapef_grad_stack = jnp.array(
#     [
#         [
#             [-9.000001, -5.0, 0.0],
#             [-0.99999905, 5.0, 0.0],
#             [9.000001, -5.0, 0.0],
#             [0.99999905, 5.0, 0.0],
#         ],
#         [[-0.0, -0.0, 0.0], [-0.0, 0.0, 0.0], [0.0, -0.0, 0.0], [0.0, 0.0, 0.0]],
#     ]
# )

# np.testing.assert_allclose(
#     expected_shapef_grad_stack, shapefunction.shapef_grad_stack
# )


Ignore the UserWarning from, the behavior is intended and expected.


/tmp/ipykernel_111774/1499691849.py:4: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  config = hdx.MPMConfig(
/tmp/ipykernel_111774/1499691849.py:14: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  grid = hdx.GridStencilMap(config)


In [3]:
from jax import make_jaxpr

print(make_jaxpr(shapefunction.update_from_particles)(grid, particles))

{ lambda ; a:i32[121] b:i32[121] c:i32[121] d:i32[2] e:i32[2] f:i32[2] g:f32[2,2]
    h:f32[2,2] i:f32[2,2] j:f32[2] k:f32[2] l:f32[] m:f32[2,3,3] n:f32[2,3,3] o:f32[2,3,3]. let
    
  in () }


In [7]:
hdx.get_pressure_stack(particles.stress_stack)

make_jaxpr(hdx.get_pressure_stack)(particles.stress_stack)

{ lambda ; a:f32[2,3,3]. let
    b:f32[2] = pjit[
      name=trace
      jaxpr={ lambda ; c:f32[2,3,3]. let
          d:i32[3,3] = iota[dimension=0 dtype=int32 shape=(3, 3) sharding=None] 
          e:i32[3,3] = iota[dimension=1 dtype=int32 shape=(3, 3) sharding=None] 
          f:i32[3,3] = add d 0
          g:bool[3,3] = eq f e
          h:f32[3,3] = broadcast_in_dim[broadcast_dimensions=() shape=(3, 3)] 0.0
          i:f32[2,3,3] = pjit[
            name=_where
            jaxpr={ lambda ; j:bool[3,3] k:f32[2,3,3] l:f32[3,3]. let
                m:bool[2,3,3] = broadcast_in_dim[
                  broadcast_dimensions=(1, 2)
                  shape=(2, 3, 3)
                ] j
                n:f32[2,3,3] = broadcast_in_dim[
                  broadcast_dimensions=(1, 2)
                  shape=(2, 3, 3)
                ] l
                o:f32[2,3,3] = select_n m n k
              in (o,) }
          ] g c h
          p:f32[2] = reduce_sum[axes=(1, 2)] i
        in (p,) }
    ] a
 